In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True, precision=2) #permet d'imprimer les array avec juste 2 chiffres et sans notation scientifique.
from openfisca_matplotlib import graphs
from datetime import date  # module nécessaire pour la définition des dates, dont notamment les dates de naissances

from openfisca_core import periods
import openfisca_france    # module décrivant le système socio-fiscal français

TaxBenefitSystem = openfisca_france.init_country()  # Initialisation de la classe décrivant le système socio-fiscal français
tax_benefit_system = TaxBenefitSystem()  # Création d'une instance du système socio-fiscal français 
#tax_benefit_cesthra = Reform



In [17]:
salaire_maximal = 1000000
year = 2014

# Axes non fonctionnel en multiannuel

Solution proposée vue en cours ne marche plus...

In [18]:
number_of_year = 3
simulation = tax_benefit_system.new_scenario().init_single_entity(
    axes = [
        dict(
            count = 5,
            min = 0 * number_of_year,
            max = 1000000 * number_of_year,
            name = 'salaire_de_base',
            ),
        ],
    period = '2010:{}'.format(number_of_year),
    parent1 = dict(
        birth = date(1980, 1, 1),
#       salaire_de_base = {"2013": salaire_maximal, "2014": salaire_maximal}
        ),
     menage = dict(
        loyer = 1000 * 3,
        so = 4,
        ),
    ).new_simulation(debug = True)


In [19]:
print simulation.calculate('sal', '2010')
print simulation.calculate('sal', '2011')
print simulation.calculate('sal', '2012')
print simulation.calculate('sal', '2013')

[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.]


Nouvelle solution proposée : 

In [20]:
def homme_celibataire_tehr(nombre_enfants = 0):
    simulation = tax_benefit_system.new_scenario().init_single_entity(
        axes = [
            dict(
                count = 5,
                min = 0,
                max = salaire_maximal,
                name = 'salaire_de_base',
                ),
            ],
        period = year,
        parent1 = dict(
            birth = date(1980, 1, 1),
            salaire_de_base = {"2013": salaire_maximal, "2014": salaire_maximal}
            ),
        menage = dict(
            loyer = 1000 * 3,
            so = 4,
            ),
        ).new_simulation(debug = True)
    return simulation

In [24]:
print homme_celibataire_tehr(nombre_enfants = 0).calculate('salaire_de_base', '2014')
print homme_celibataire_tehr(nombre_enfants = 0).calculate('salaire_de_base', '2013')

[       0.   250000.   500000.   750000.  1000000.]
[ 1000000.  1000000.  1000000.  1000000.  1000000.]


Avant la fonction axe faisait varier l'ensemble des revenus de chaque année de la simulation, maintenant ça ne fait plus que varier l'année de la simulation. Ça serait bien de pouvoir faire comme avant. Autrement les riches touchent du rsa si on met 0 et les pauvres en touchent pas si on met sal_max...

### Calcul de variables :

Certaines variables ne peuvent plus être calculées, exemple du taux de cotisation sociales. C'est tiré texto du deuxième cours que tu as fait.

In [27]:
cotsoc= abs(
        homme_celibataire_tehr().calculate('cotisations_salariales','2013') 
        + homme_celibataire_tehr().calculate('csgsald','2013') 
        + homme_celibataire_tehr().calculate('csgsali','2013')
        + homme_celibataire_tehr().calculate('crdssal','2013'))

AssertionError: Requested period 2013 differs from 2013-01 returned by variable csgsald

J'ai essayé de mettre différentes formes de périodes avec from base import periods, pas réussi à faire tourner. Je pense que ça vient du commit de manu ou il simplifie les calculs des périodes pour certaines variables ( Remove implicit calculations of periods from OpenFisca-Core (debc00a07ef9ca1a4c0de84ecffb2a206b946d18)) 

## Variables disparues :

Certaines variables ont été supprimées, renomées, ou autre. Il y a mismatch entre openfisca.fr/variables et les variables sur le notebook 

## Commentaires :

Je sais que si on est pas content il suffit de coder ce que l'on souhaite avoir :p . Mais rendre totalement explicite les périodes dans init_single_entity qui est censée être une classe orientée 'utilisateur' plutôt que dévelopeur ça rajoute beaucoup de choses. Peut être qu'il faudrait créer easy_init_single_entity et easy_calculate la première qui répartierai le revenu sur l'ensemble des années pour le test case, et la seconde qui calculerai une variable pour la dernière année de la simulation avec la période adéquate.

## Elements de réponse

 - l'argument period de inti_single_entity va définir la période par défaut de la simulation, c'est à dire celle qui sera utilisée par calculate si on ne précise pas la période
 - calculate ne peut calculer une variable que si la période demandée est exactement la période de la variable calculée
 - si ces ceux périodes ne coincident pas il faut utiliser calculate_add et calculate_divide
 - on ne peut plus utiliser les objets period de "size" plus grand que 1 ("2010:4" est de taille 4) dans init_single_entity

Un petit example illustratif

In [37]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    axes = [
        dict(
            count = 5,
            min = 0 * number_of_year,
            max = 1000000 * number_of_year,
            name = 'salaire_de_base',
            ),
        ],
    period = '2014',
    parent1 = dict(
        birth = date(1980, 1, 1),
        ),
     menage = dict(
        loyer = 1000 * 3,
        so = 4,
        ),
    ).new_simulation(debug = True)

In [40]:
print simulation.calculate('salaire_de_base', period = "2013")
print simulation.calculate('salaire_de_base', period = "2014")


[ 0.  0.  0.  0.  0.]
[       0.   750000.  1500000.  2250000.  3000000.]


In [41]:
simulation.calculate('csgsald') # erreur car csgsald est calculé mensuellement comme indiqué dans le message ci-dessous

AssertionError: Requested period 2014 differs from 2014-01 returned by variable csgsald

In [42]:
simulation.calculate_add('csgsald') 

array([-0., -0., -0., -0., -0.], dtype=float32)

On a zéro car on n'a pas définit les salaires mensuels

In [43]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = '2014',
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = {
            "2014-01": 10000,
            "2014-02": 10000,
            }
        ),
     menage = dict(
        loyer = 1000 * 3,
        so = 4,
        ),
    ).new_simulation(debug = True)

In [46]:
simulation.calculate('csgsald', period = "2014-01") 

array([-501.08], dtype=float32)

In [48]:
simulation.calculate_add('csgsald', period = "2014") 

array([-1002.15], dtype=float32)